## Workflow

1. Query the STAC API to retrieve an item from the TDM30 EDEM collection
2. Generate map tiles through the raster API endpoint
3. Render the tiles on an interactive map with `folium`

In [1]:
import requests
import folium

## Configure API Endpoints

Discover available datasets through:

* **Programmatically**: See the `list-collections.ipynb` notebook
* **JSON API**: https://api.dev.veda.grss.cloud/stac/collections

In [2]:
STAC_API_URL = "https://api.dev.veda.grss.cloud/stac"
RASTER_API_URL = "https://api.dev.veda.grss.cloud/raster"

collection_id = "TDM30_EDEM"

## Retrieve Collection Metadata

Fetch the collection details to understand the available data and its properties.

In [3]:
response = requests.get(f"{STAC_API_URL}/collections/{collection_id}")
assert response.ok, response.text

collection = response.json()
collection

{'id': 'TDM30_EDEM',
 'type': 'Collection',
 'links': [{'rel': 'items',
   'type': 'application/geo+json',
   'href': 'https://api.dev.veda.grss.cloud/stac/collections/TDM30_EDEM/items'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://api.dev.veda.grss.cloud/stac/'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://api.dev.veda.grss.cloud/stac/'},
  {'rel': 'self',
   'type': 'application/json',
   'href': 'https://api.dev.veda.grss.cloud/stac/collections/TDM30_EDEM'},
  {'rel': 'items',
   'href': 'https://api.dev.veda.grss.cloud/stac/collections/TDM30_EDEM/items',
   'type': 'application/geo+json'},
  {'rel': 'http://www.opengis.net/def/rel/ogc/1.0/queryables',
   'href': 'https://api.dev.veda.grss.cloud/stac/collections/TDM30_EDEM/queryables',
   'type': 'application/schema+json',
   'title': 'Queryables'},
  {'rel': 'license',
   'href': 'https://geoservice.dlr.de/resources/licenses/tdm30-edited/License_for_the_Utilization_of_TanDEM-X_30m

## Query for a Specific Item

Use the STAC search endpoint to find items within the collection. For digital elevation models, we typically query by spatial extent.

In [4]:
# Query items from the collection
response = requests.get(
    f"{STAC_API_URL}/collections/{collection_id}/items",
    params={"limit": 10}
)

assert response.ok, response.text

items = response.json()["features"]
print(f"Found {len(items)} items")

Found 10 items


Examine the structure of an item to understand the available assets:

In [5]:
item = items[0]
item

{'id': 'TDM1_EDEM_10_N62W008_EDEM_W84_cog',
 'bbox': [-8.000277777777777,
  61.999861111111116,
  -5.999722222222221,
  63.00013888888889],
 'type': 'Feature',
 'links': [{'rel': 'collection',
   'type': 'application/json',
   'href': 'https://api.dev.veda.grss.cloud/stac/collections/TDM30_EDEM'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://api.dev.veda.grss.cloud/stac/collections/TDM30_EDEM'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://api.dev.veda.grss.cloud/stac/'},
  {'rel': 'self',
   'type': 'application/geo+json',
   'href': 'https://api.dev.veda.grss.cloud/stac/collections/TDM30_EDEM/items/TDM1_EDEM_10_N62W008_EDEM_W84_cog'},
  {'title': 'Map of Item',
   'href': 'https://api.dev.veda.grss.cloud/raster/collections/TDM30_EDEM/items/TDM1_EDEM_10_N62W008_EDEM_W84_cog/WebMercatorQuad/map?assets=dem&rescale=-500%2C8000&colormap_name=terrain',
   'rel': 'preview',
   'type': 'text/html'}],
 'assets': {'dem': {'href': 's3://ieee-grs

## Determine Visualization Parameters

Extract statistics from the item to set appropriate rescale values for rendering.

In [6]:
# Get the primary asset key
asset_keys = list(item['assets'].keys())
print(f"Available assets: {asset_keys}")

# Select the data asset (exclude metadata assets like 'rendered_preview')
data_asset = [k for k in asset_keys if k not in ['rendered_preview']][0]
print(f"Using asset: {data_asset}")

Available assets: ['dem', 'rendered_preview']
Using asset: dem


In [7]:
# Set elevation rescale values (typical range for terrain)
rescale_values = (0, 3000)

## Generate Map Tiles

Request tile metadata from the raster API to visualize the elevation data.

In [8]:
tile_matrix_set_id = "WebMercatorQuad"
colormap_name = "terrain"

response = requests.get(
    f"{RASTER_API_URL}/collections/{collection_id}/items/{item['id']}/{tile_matrix_set_id}/tilejson.json",
    params={
        "assets": data_asset,
        "colormap_name": colormap_name,
        "rescale": f"{rescale_values[0]},{rescale_values[1]}"
    }
)

assert response.ok, response.text

tiles = response.json()
tiles

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://api.dev.veda.grss.cloud/raster/collections/TDM30_EDEM/items/TDM1_EDEM_10_N62W008_EDEM_W84_cog/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=dem&colormap_name=terrain&rescale=0%2C3000'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-8.000277777777777,
  61.999861111111116,
  -5.999722222222221,
  63.00013888888889],
 'center': [-6.999999999999999, 62.5, 0]}

## Display the Elevation Map

Create an interactive map using `folium` to visualize the terrain data.

In [9]:
# Calculate center from item bounds
bbox = item['bbox']
center_lat = (bbox[1] + bbox[3]) / 2
center_lon = (bbox[0] + bbox[2]) / 2

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=8,
    tiles="CartoDB positron"
)

folium.TileLayer(
    tiles=tiles["tiles"][0],
    attr="GRSS VEDA",
    opacity=0.7,
    overlay=True,
    name="Elevation"
).add_to(m)

folium.LayerControl().add_to(m)

m